In [14]:
import tensorflow as tf
import numpy as np
import os

In [15]:
shakespeare_text = None
with open('input.txt') as f:
    shakespeare_text = f.read()

In [16]:
vocab = sorted(set(shakespeare_text))
print ('The number of unique characters in the corpus is', len(vocab))
print('A slice of the unique characters set:\n', vocab[:10])

The number of unique characters in the corpus is 65
A slice of the unique characters set:
 ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3']


In [18]:
char2idx = {u:i for i, u in enumerate(vocab)}
# Make a copy of the unique set elements in NumPy array format for later use in the decoding the predictions
idx2char = np.array(vocab)
# Vectorize the text with a for loop
text_as_int = np.array([char2idx[c] for c in shakespeare_text])

In [19]:

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) 
# for i in char_dataset.take(5): 
#   print(i.numpy())
seq_length = 100 # The max. length for single input
# examples_per_epoch = len(text)//(seq_length+1) # double-slash for “floor” division
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) 
# for item in sequences.take(5): 
#   print(repr(''.join(idx2char[item.numpy()])))

In [20]:

def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [21]:
BUFFER_SIZE = 10000 # TF shuffles the data only within buffers

BATCH_SIZE = 64 # Batch size

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


In [22]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units = 1024

In [23]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [25]:
model = build_model(
    vocab_size = len(vocab), # no. of unique characters
    embedding_dim=embedding_dim, # 256
    rnn_units=rnn_units, # 1024
    batch_size=BATCH_SIZE)  # 64 for the traning

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru_2 (GRU)                 (64, None, 1024)          3938304   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [26]:

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# example_batch_loss  = loss(target_example_batch, example_batch_predictions)
# print("Prediction shape: ", example_batch_predictions.shape, " (batch_size, sequence_length, vocab_size)")
# print("scalar_loss:      ", example_batch_loss.numpy().mean())

model.compile(optimizer='adam', loss=loss)

In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [45]:
for a in dataset.as_numpy_iterator:
    print(a)

TypeError: 'method' object is not iterable

In [28]:

EPOCHS = 30
history = model.fit(dataset, 
                    epochs=EPOCHS, 
                    callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 267s 2s/step - loss: 2.6478
Epoch 2/30
172/172 [==============================] - 267s 2s/step - loss: 1.9499
Epoch 3/30
172/172 [==============================] - 271s 2s/step - loss: 1.6831
Epoch 4/30
172/172 [==============================] - 272s 2s/step - loss: 1.5377
Epoch 5/30
172/172 [==============================] - 276s 2s/step - loss: 1.4512
Epoch 6/30
172/172 [==============================] - 275s 2s/step - loss: 1.3928
Epoch 7/30
172/172 [==============================] - 267s 2s/step - loss: 1.3475
Epoch 8/30
172/172 [==============================] - 269s 2s/step - loss: 1.3082
Epoch 9/30
172/172 [==============================] - 272s 2s/step - loss: 1.2738
Epoch 10/30
172/172 [==============================] - 268s 2s/step - loss: 1.2417
Epoch 11/30
172/172 [==============================] - 262s 2s/step - loss: 1.2096
Epoch 12/30
172/172 [==============================] - 270s 2s/step - loss: 1.1772
Epoch 13/30
1

In [36]:
def generate_text(model, num_generate, temperature, start_string):
  input_eval = [char2idx[s] for s in start_string] # string to numbers (vectorizing)
  input_eval = tf.expand_dims(input_eval, 0) # dimension expansion
  text_generated = [] # Empty string to store our results
  model.reset_states() # Clears the hidden states in the RNN

  for i in range(num_generate): #Run a loop for number of characters to generate
    predictions = model(input_eval) # prediction for single character
    predictions = tf.squeeze(predictions, 0) # remove the batch dimension

    # using a categorical distribution to predict the character returned by the model
    # higher temperature increases the probability of selecting a less likely character
    # lower --> more predictable
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # The predicted character as the next input to the model
    # along with the previous hidden state
    # So the model makes the next prediction based on the previous character
    input_eval = tf.expand_dims([predicted_id], 0) 
    # Also devectorize the number and add to the generated text
    text_generated.append(idx2char[predicted_id]) 

  return (start_string + ''.join(text_generated))

In [38]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_3 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_2 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [39]:
generated_text = generate_text(
                    model, 
                    num_generate=500, 
                    temperature=1, 
                    start_string=u"ROMEO")
print(generated_text)

ROMEO:
Come, sir, and I dare not say he: 'fore mestrum lies you into my house;
For both offen, ere they give;
And, for I am a manner of mercy. And his wonder;
Nor what am too much first and what time had made
That thou art.

AUFIDIUS:
You have done enough, nor worse: is't my speech and
Willought to my foe till thou do: abother,
If he hath lost advanced. O, whilst I dur
boy, thou art a tall fellow, and his
school'd my wife!

ANTONIO:
His noble country's reasons of the worst
Of Sochaid to be unclar, ev


In [29]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

In [30]:
tokenizer.texts_to_sequences(["First"]), tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]]), tokenizer.texts_to_sequences([" "])

([[20, 6, 9, 8, 3]], ['f i r s t'], [[1]])

In [31]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

max_id, dataset_size

(39, 1)

In [32]:
# encode the full text so each character is represented by its ID 
# (we subtract 1 to get IDs from 0 to 38, rather than from 1 to 39)

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [33]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [34]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead 
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [35]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset 

<FlatMapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int64, name=None)>

In [10]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [11]:
dataset.counter

<function tensorflow.python.data.ops.dataset_ops.DatasetV2.counter(start=0, step=1, dtype=tf.int64, name=None)>

In [12]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))])
    
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [13]:
history = model.fit(dataset, epochs=20)

Epoch 1/20


ValueError: in user code:

    File "/Users/piyushr/workspace/personal/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/piyushr/workspace/personal/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/piyushr/workspace/personal/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/piyushr/workspace/personal/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/Users/piyushr/workspace/personal/venv/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/piyushr/workspace/personal/venv/lib/